In [ ]:
import utils
from utils import *
BATCH_SIZE = hyper_parameters['BATCH_SIZE']
PREPROCESS = True
FLIP = False

#### Pokemon Dataloader

In [ ]:
if PREPROCESS:
    try:
        os.system("rm ./Pokemon/flipped_*")
    except:
        print("No flipped images")

    create_csv()
    if FLIP:
        pokelist = pd.read_csv("./PokeList", header=None)
        for i in range(851):
            pokemon=np.fliplr(mpimg.imread("./Pokemon/"+ pokelist[i][0]))
            plt.imsave("./PokemonFlipped/flipped_" + pokelist[i][0][:-4] + ".jpg", pokemon)
        os.system("mv ./PokemonFlipped/* ./Pokemon/")

In [ ]:
if PREPROCESS:
    create_csv()
pokeSet = PokeSet("./Pokemon/")
pokeLoader = Utils.DataLoader(dataset = pokeSet, shuffle = True, batch_size = BATCH_SIZE)

#### Train Network

In [ ]:
# Training
label = "L50"
row = 2
col = 3

losses = []
bces = []
kls = []
def train(model,optimizer,dataloader, epoch, max_epochs=5000000):
    
    for _ in range(max_epochs):
        epoch += 1
        for images in dataloader:
            image_in = images.permute(0,3,1,2)
            x_in = Variable(image_in.float()).cuda()
            optimizer.zero_grad()
#             x_out, z_mu, z_logvar = model(x_in)
            loss, bce, kl = model.calculate_loss(x_in)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            bces.append(bce.item())
            kls.append(kl.item())
            
        if epoch % 1 == 0 and epoch != 0:
            save_file = label + "_epoch_{:06d}".format(epoch) + '.pth'
            torch.save({
                'epoch': epoch,
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
                'losses' : losses
            }, "checkpoints/" + save_file)
                
        clear_output(wait=True)
        if epoch % 1 == 0:
            with torch.no_grad():
                pokePlot(images, model, row, col, epoch)
        print(epoch, 'Loss: {:3f}'.format(loss.item()))
        
    return losses, bces, kls
try:
    net, epoch, losses, optimizer = load_checkpoint("./checkpoints/")
except:
    net = Net().cuda()
    optimizer = optim.Adam(net.parameters(), lr = 0.001)
    epoch = 0
    
train_losses, bces, kls = train(net,optimizer,pokeLoader, epoch)
# plt.figure(figsize=(10,5))
# plt.plot(train_losses)
# plt.show()

# plt.figure(figsize=(10,5))
# plt.plot(bces)
# plt.show()

# plt.figure(figsize=(10,5))
# plt.plot(kls)
# plt.show()

In [ ]:
pokemon = mpimg.imread("./Pokemon/squirtle.jpg")
if pokemon.shape[-1] == 4:
    pokemon = pokemon[:,:,:3]
pokemon = cv2.resize(pokemon, (RESIZE,RESIZE))
pokemon = (pokemon - np.min(pokemon))/np.max(pokemon - np.min(pokemon))

# pokemon = np.rot90(pokemon, axes=[0,1]).copy()
# plt.imshow(pokemon)
# print(pokemon)

image = torch.from_numpy(pokemon).unsqueeze(0)
image = image[0,:,:,:].unsqueeze(0)
plt.imshow((image.squeeze().numpy()))
plt.show()
image = image.permute(0,3,1,2)
x_out, z_mean, z_logvar = net(Variable(image.float()).cuda())
x_out = x_out.permute(0,2,3,1)
plt.imshow(x_out.data.cpu().squeeze().numpy())
plt.show()

In [ ]:
def sweep(image, dim, min_range, max_range, step):
#     plt.imshow((image.squeeze().numpy()))
#     plt.show()
    image = image.permute(0,3,1,2)
    z_mean, z_logvar = net.encoder(Variable(image.float()).cuda())
    z = net.latent(z_mean,z_logvar)
    for i, sw in enumerate(range(min_range, max_range, step)):
        z[0][dim] = sw/100
        x_out = net.decoder(z)
        x_out = x_out.permute(0,2,3,1)
        im = (np.floor(x_out.data.cpu().squeeze().numpy() * 255)).astype(np.uint8)
        plt.imsave("./sweep/" + str(dim) + "/" + "{0:03d}".format(i) + ".png", im)
    generate_animation("./sweep/" + str(dim) + "/", dim, (max_range - min_range)//step)

In [ ]:
def generate_animation(path, dim, num):
    images = []
    for e in range(num):
        img_name = path + "{0:03d}".format(e) + '.png'
        images.append(mpimg.imread(img_name))
    imageio.mimsave("./sweep/gif/" + 'dim' + str(dim) + '_generate_animation.gif', images, fps=30)


In [ ]:
image = image.permute(0,2,3,1)
os.system("rm -r ./sweep/*")
os.system("mkdir ./sweep/gif/")
for i in range(LATENT_DIM):
    os.system("mkdir ./sweep/"+str(i))
    sweep(image, i, -300, 300, 10)

In [ ]:
def interpolate_linear(path1, path2, savepath, STEP):
    pokemon = mpimg.imread(path1)
    if pokemon.shape[-1] == 4:
        pokemon = pokemon[:,:,:3]
    pokemon = cv2.resize(pokemon, (RESIZE,RESIZE))
    pokemon = (pokemon - np.min(pokemon))/np.max(pokemon - np.min(pokemon))
    image = torch.from_numpy(pokemon).unsqueeze(0)

    image = image[0,:,:,:].unsqueeze(0)
    image = image.permute(0,3,1,2)
    z_mean, z_logvar = net.encoder(Variable(image.float()).cuda())
    z1 = net.latent(z_mean, z_logvar)
    ss = net.decoder(z1)
    ss = ss.permute(0,2,3,1)
    plt.imshow(ss.data.cpu().squeeze().numpy())
    plt.show()

    pokemon = mpimg.imread(path2)
    if pokemon.shape[-1] == 4:
        pokemon = pokemon[:,:,:3]
    pokemon = cv2.resize(pokemon, (RESIZE,RESIZE))
    pokemon = (pokemon - np.min(pokemon))/np.max(pokemon - np.min(pokemon))
    image = torch.from_numpy(pokemon).unsqueeze(0)

    image = image[0,:,:,:].unsqueeze(0)
    image = image.permute(0,3,1,2)
    z_mean, z_logvar = net.encoder(Variable(image.float()).cuda())
    z2 = net.latent(z_mean, z_logvar)
    ss = net.decoder(z2)
    ss = ss.permute(0,2,3,1)
    plt.imshow(ss.data.cpu().squeeze().numpy())
    plt.show()
    step = (z2-z1)/STEP

    for i in range(STEP+1):
        ss = net.decoder(z1 + i*step)
        ss = ss.permute(0,2,3,1)
        plt.imsave(savepath + "{0:03d}".format(i) + ".png", ss.data.cpu().squeeze().numpy())

In [ ]:
def generate_animation_interpol(path, name, num):
    images = []
    for e in range(num):
        img_name = path + "{0:03d}".format(e) + '.png'
        images.append(mpimg.imread(img_name))
    imageio.mimsave("./interpolate/gif/" + name + '_generate_animation.gif', images, fps=30)


In [ ]:
STEP = 100
poke1 = "./Pokemon/venusaur.jpg"
poke2 = "./Pokemon/flipped_venusaur.jpg"
label = poke1[10:-4] + "_" + poke2[10:-4]

i = 0
while(1):
    if not (os.path.exists("./interpolate/" + label)):
        os.system("mkdir " + "./interpolate/" + label)
        print(label)
        break
    else:
        i += 1
        if label [-2] == "_":
            label = label[:-2]
        label = label + "_" + str(i)
        
interpolate_linear(poke1, poke2, "./interpolate/" + label + "/", STEP)
generate_animation_interpol("./interpolate/" + label + "/", label, STEP)

In [ ]:
# Visualize test data encodings on the latent space
def visualize_encoder(model,dataloader):
    z_means_x, z_means_y, all_labels = [], [], []
    
    for images in iter(dataloader):
        images = images.permute(0,3,1,2)
        z_means,_ = model.encoder(Variable(images.float()))
        z_means_x = np.append(z_means_x,z_means[:,0].data.numpy())
        z_means_y = np.append(z_means_y,z_means[:,1].data.numpy())
#         all_labels = np.append(all_labels,labels.numpy())
        
    plt.figure(figsize=(6.5,5))
    plt.scatter(z_means_x,z_means_y,cmap='inferno')
    plt.show()

visualize_encoder(net,pokeLoader)

In [ ]:
# Visualize moving average of losses
def visualize_losses_moving_average(losses,window=50,boundary='valid'):
    mav_losses = np.convolve(losses,np.ones(window)/window,boundary)
    corrected_mav_losses = np.append(np.full(window-1,np.nan),mav_losses)
    plt.figure(figsize=(10,5))
    plt.plot(losses)
    plt.plot(corrected_mav_losses)
    plt.ylim(ylim)
    plt.show()

visualize_losses_moving_average(train_losses)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, _ = data

        # wrap them in Variable
        inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        x_out, z_mean, z_logvar = net(inputs)
        loss = criterion(x_out, inputs, z_mean, z_logvar)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

#### Testing Network

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))

#Predict with network
outputs, _, _ = net(Variable(images))

# _, predicted = torch.max(outputs.data, 1)

# print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                               for j in range(4)))


imshow(torchvision.utils.make_grid(outputs.data))

In [ ]:
#Test network accuracy

correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs, _, _ = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

#### RUN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from scipy.stats import norm
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import make_grid as make_image_grid
from tqdm import tnrange

torch.manual_seed(2017) # reproducability
sns.set_style('dark')
%matplotlib inline

In [ ]:
# Model
class VAE(nn.Module):
    def __init__(self,latent_dim=20,hidden_dim=500):
        super(VAE,self).__init__()
        self.fc_e = nn.Linear(784,hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim,latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim,latent_dim)
        self.fc_d1 = nn.Linear(latent_dim,hidden_dim)
        self.fc_d2 = nn.Linear(hidden_dim,784)
            
    def encoder(self,x_in):
        x = F.relu(self.fc_e(x_in.view(-1,784)))
        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)
        return mean, logvar
    
    def decoder(self,z):
        z = F.relu(self.fc_d1(z))
        x_out = F.sigmoid(self.fc_d2(z))
        return x_out.view(-1,1,28,28)
    
    def sample_normal(self,mean,logvar):
        # Using torch.normal(means,sds) returns a stochastic tensor which we cannot backpropogate through.
        # Instead we utilize the 'reparameterization trick'.
        # http://stats.stackexchange.com/a/205336
        # http://dpkingma.com/wordpress/wp-content/uploads/2015/12/talk_nips_workshop_2015.pdf
        sd = torch.exp(logvar*0.5)
        e = Variable(torch.randn(sd.size())) # Sample from standard normal
        z = e.mul(sd).add_(mean)
        return z
    
    def forward(self,x_in):
        z_mean, z_logvar = self.encoder(x_in)
        z = self.sample_normal(z_mean,z_logvar)
        x_out = self.decoder(z)
        return x_out, z_mean, z_logvar

model = VAE()

In [ ]:
# Loss function
def criterion(x_out,x_in,z_mu,z_logvar):
    bce_loss = F.binary_cross_entropy(x_out,x_in,size_average=False)
    kld_loss = -0.5 * torch.sum(1 + z_logvar - (z_mu ** 2) - torch.exp(z_logvar))
    loss = (bce_loss + kld_loss) / x_out.size(0) # normalize by batch size
    return loss

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Data loaders
trainloader = DataLoader(
    MNIST(root='./MNIST',train=True,download=True,transform=transforms.ToTensor()),
    batch_size=128,shuffle=True)
testloader = DataLoader(
    MNIST(root='./MNIST',train=False,download=True,transform=transforms.ToTensor()),
    batch_size=128,shuffle=True)

In [ ]:
# Training
def train(model,optimizer,dataloader,epochs=15):
    losses = []
    for epoch in range(epochs):
        for images,_ in dataloader:
            x_in = Variable(images)
            optimizer.zero_grad()
            x_out, z_mu, z_logvar = model(x_in)
            loss = criterion(x_out,x_in,z_mu,z_logvar)
            loss.backward()
            optimizer.step()
            losses.append(loss.data[0])
        print('Loss: {:4f}'.format(loss.data[0]))
    return losses

train_losses = train(model,optimizer,trainloader)
plt.figure(figsize=(10,5))
plt.plot(train_losses)
plt.show()